In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sys

In [ ]:
sys.path.append("../")

In [ ]:
def is_sequence_a_peak(model,sequence,alphabet="AGTC"):
    neighbor=[s for s in sequence]
    sequence_fitness=model.get_fitness(sequence)
    for position in range(len(sequence)):
        for aa in alphabet:
            if aa!=sequence[position]:
               neighbor[position]=aa
               neighbor_string="".join(neighbor)
               if sequence_fitness<model.get_fitness(neighbor_string):
                  return 0
               neighbor=[s for s in sequence]
    return 1

In [ ]:
def get_all_peaks(landscape):
    peaks=set()
    for seq in landscape.sequences:
        if is_sequence_a_peak(landscape,seq):
           peaks.add(seq)
    return peaks

In [ ]:
from models.Ground_truth_oracles.TF_binding_landscape_models import *

In [ ]:
landscape_constructor=TF_binding_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test = ['SIX6_REF_R1'])

In [ ]:
landscape_constructor=landscape_constructor.generate_from_loaded_landscapes()
landscape=next(landscape_constructor)

In [ ]:
peaks= get_all_peaks(landscape["landscape_oracle"])

In [ ]:
len(peaks)

In [ ]:
data=pd.read_csv("../simulations/evaluations_test_six6/consistency_robustness_independence/Greedy_mu1_tr0.05_r0.1_rho1.csv")

In [ ]:
data1=pd.read_csv("../simulations/evaluations_test_six6/consistency_robustness_independence/MLWFG_mu1_r0.1_rho1_beta100.csv")

In [ ]:
data2=pd.read_csv("../simulations/evaluations_test_six6_1/consistency_robustness_independence/Greedy_mu1_tr0.05_r0.1_rho1.csv")

In [ ]:
data.tail()

In [ ]:
data.model_type.unique()

In [ ]:
data.columns

In [ ]:
data['id'].unique()

In [ ]:
sub_data=data[(data['virtual_screen']==50)]# &(data['id']=='48eb64e4-3f1b-11ea-8fd5-f218984ddad3')]#[data.model_type=="nn_arch=CNNa_hd100_f50"]

In [ ]:
sub_data1= data1[(data1['virtual_screen']==50)]

In [ ]:
sub_data2= data2[(data2['virtual_screen']==10)]

In [ ]:
plt.figure(figsize=(12,5))
sns.lineplot(x="batch",y="true_score",hue="model_type",data=sub_data2 )
plt.legend(bbox_to_anchor=(1,1))

In [ ]:
sub_data=data[data.model_type=='nn_arch=CNNa_hd100_f50']
sub_data1=data1[data.model_type=='nn_arch=CNNa_hd100_f50']

In [ ]:
sub_data1=data1[data1.model_type=='nn_arch=CNNa_hd100_f50']

In [ ]:
sub_data2=data2[data2.model_type.str.startswith('ENS_nn_arch=CNNa_hd100_f50')]

In [ ]:
all_seqs=sub_data2[sub_data2.start_id=="TF0"].sequence.unique()

In [ ]:
count=0
found_peaks=[]
for seq in all_seqs:
    if seq in peaks:
       found_peaks.append(seq) 
       count+=1
count

In [ ]:
sns.distplot([landscape["landscape_oracle"].get_fitness(x) for x in peaks])
sns.distplot([landscape["landscape_oracle"].get_fitness(x) for x in found_peaks])

In [ ]:
def get_top_sequence_per_batch(data):
    top_per_batch=[]
    for i in data.batch.unique():
        sub_data=data[data.batch==i] #must also group by model type and landscape id
        #print (sub_data.true_score.max())
        top_per_batch.append(sub_data.true_score.max())
    return (top_per_batch)

In [ ]:
def compute_cumulative_outcomes(data): 
    outcomes=pd.DataFrame(columns=["landscape_id","start_id","model_type","batch","max_fitness","cum_max_fitness"])
    for landscape in data.landscape_id.unique():
        if landscape==str(-1): #or start_id==str(-1):
            continue
        print(landscape)
        valid_start_ids=[x for x in data.start_id.unique()]# if landscape[landscape.index("L"): landscape.index("R")] in x]
        for start_id in valid_start_ids:
            for noisy_model in data.model_type.unique():
                sub_data = data[(data.landscape_id==landscape)&\
                               (data.start_id==start_id)&\
                               (data.model_type==noisy_model)] 
                cum_fit=0
                for batch, max_fitness in enumerate(get_top_sequence_per_batch(sub_data)):
                    if max_fitness>cum_fit:
                       cum_fit = max_fitness 
                    outcomes = outcomes.append(pd.DataFrame.from_records([{"landscape_id":landscape,\
                                          "start_id":start_id,\
                                          "model_type": noisy_model,\
                                          "batch": batch+1, \
                                          "max_fitness": max_fitness,\
                                          "cum_max_fitness": cum_fit}])) 
                    
    return outcomes

In [ ]:
outcomes=compute_cumulative_outcomes(sub_data)
outcomes1=compute_cumulative_outcomes(sub_data1)
outcomes1=compute_cumulative_outcomes(sub_data2)

outcomes.head()

In [ ]:
plt.figure(figsize=(14,5))

#sns.lineplot(x="batch",y="cum_max_fitness",hue="model_type",data= outcomes)
sns.lineplot(x="batch",y="cum_max_fitness",hue="model_type",data= outcomes1, color="r")

In [ ]:
("_").join(set(['a','a','a']))